In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from scipy.misc import imsave
import matplotlib.image as mpimg
import cv2
from PIL import Image

Using TensorFlow backend.


In [15]:
df = pd.read_csv('data/train.csv')
X_df = df['ImageId']
y_df = df['Malignant']
X = X_df.values
y = y_df.values

In [1]:
def build_matrix(X,folder):
    # load images
    M = []
    for i in range(0, X.shape[0]):
        image = X[i]
        im = plt.imread(folder + '/'  + str(image) + '.jpg')
        #link = 'data/scaled' + '/'  + str(image) + '_segmentation.jpg'
        img_down = resize(im,(224,224), mode='reflect')
        #plt.imsave(image+'.jpg',img_down,cmap='gray')
        M.append(img_down)
    return np.asarray(M)

In [17]:
M = build_matrix(X,folder='data/im')

C:\Users\ASUS\Anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [55]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [57]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

batch_size = 16

In [11]:
M.shape

(700, 256, 256)

In [18]:
train_images = M[0:500]
#train_images = train_images.astype('float32') / 255
train_labels = y[0:500]
test_images = M[500:]
#test_images = test_images.astype('float32') / 255
test_labels = y[500:]

In [49]:
from keras import models
from keras import layers



model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))



#model =  models.Sequential()
#model.add(Convolution2D(filters = 32, kernel_size = (3, 3), input_shape = (256, 256, 3),activation = 'relu'))
#model.add(MaxPooling2D(pool_size = (2, 2)))
#model.add(Convolution2D(32, 3, 3, activation = 'relu'))
#model.add(MaxPooling2D(pool_size = (2, 2)))
#model.add(Convolution2D(32, 3, 3, activation = 'relu'))
#model.add(MaxPooling2D(pool_size = (2, 2)))
#model.add(Convolution2D(32, 3, 3, activation = 'relu'))
#model.add(MaxPooling2D(pool_size = (2, 2)))
#model.add(Flatten())
#model.add(Dense(units = 128, activation = 'relu'))
#model.add(Dense(units = 1, activation = 'softmax'))


model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['accuracy'])




In [58]:
model.fit(M, y, epochs=5, batch_size=64)

Epoch 1/5
700/700 [==============================] - 77s 110ms/step - loss: 1.3159 - acc: 0.5414
Epoch 2/5
700/700 [==============================] - 66s 95ms/step - loss: 0.6767 - acc: 0.6000
Epoch 3/5
700/700 [==============================] - 66s 95ms/step - loss: 0.7088 - acc: 0.5643
Epoch 4/5
700/700 [==============================] - 66s 95ms/step - loss: 0.6808 - acc: 0.6014
Epoch 5/5
700/700 [==============================] - 67s 95ms/step - loss: 0.6966 - acc: 0.5886


In [29]:
#train_images = train_images.astype('float32') / 255
#test_images = test_images.astype('float32') / 255

#from keras.utils import to_categorical
#train_labels = to_categorical(train_labels)
#test_labels = to_categorical(test_labels)

In [31]:
#test_loss, test_acc = model.evaluate(test_images, test_labels)
#print('test_acc:', test_acc)

200/200 [==============================] - 9s 47ms/step
test_acc: 0.355


In [ ]:
df_submission = pd.read_csv('data/test.csv')
sub = build_matrix(df_submission['ImageId'].values,folder='data/im_segmentations')

In [ ]:
rslt = model.predict(sub)

In [71]:
for i in range(0,rslt.shape[0]):
    if rslt[i]>0.357:
        rslt[i] = 1
    else : 
        rslt[i] = 0

In [ ]:
i=0
for Id in df_submission['ImageId']:
    value = rslt[i] 
    df_submission.loc[df_submission['ImageId'] == Id, 'Malignant'] = value
    i=i+1
    
    
df_submission['Malignant'] = df_submission['Malignant'].astype(int) # This line is mandatory to be sure to have integer
print(df_submission.head(3))
df_submission.to_csv('data/lyoum_test.csv', index=None, sep=',', mode='w') # Save the data in the exemple_test.csv file

In [6]:
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [19]:


# Check if GPU is available
# K.tensorflow_backend._get_available_gpus()

# The paths for the training and validation images
train_path = 'data/train_scaled'
valid_path = 'data/test_scaled'

# Declare a few useful values
num_train_samples = 700
num_val_samples = 300
train_batch_size = 10
val_batch_size = 10
image_size = 224

# Declare how many steps are needed in an iteration
train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [20]:
# Set up generators
train_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    batch_size=train_batch_size)

valid_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=val_batch_size)

test_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=val_batch_size,
    shuffle=False)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [21]:
# Create a MobileNet model
mobile = keras.applications.mobilenet.MobileNet()


In [22]:


# Modify the model
# Exclude the last 5 layers of the model
x = mobile.layers[-6].output
# Add a dropout and dense layer for predictions
x = Dropout(0.25)(x)
predictions = Dense(1, activation='softmax')(x)

# Create a new model with the new outputs
model = Model(inputs=mobile.input, outputs=predictions)


In [23]:


# Freeze the weights of the layers that we aren't training (training the last 23)
for layer in model.layers[:-23]:
    layer.trainable = False

# Train the model
# Define Top2 and Top3 Accuracy
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy

In [27]:
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

# Compile the model
model.compile(Adam(lr=0.01), loss='sparse_categorical_crossentropy', metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])


In [28]:
# Declare the filepath for the saved model
filepath = "model.h5"

In [29]:
# Declare a checkpoint to save the best version of the model
checkpoint = ModelCheckpoint(filepath, monitor='val_top_3_accuracy', verbose=1,
                             save_best_only=True, mode='max')

# Reduce the learning rate as the learning stagnates
reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2,
                              verbose=1, mode='max', min_lr=0.00001)

callbacks_list = [checkpoint, reduce_lr]

# Fit the model
model.fit(M, y, epochs=5, batch_size=64)

Epoch 1/5


InvalidArgumentError: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 0 1 0 1 0 1 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0
	 [[{{node loss_2/dense_2_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}} = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _class=["loc:@train...s_grad/mul"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](loss_2/dense_2_loss/Log, loss_2/dense_2_loss/Cast)]]

In [18]:






# Evaluate the model
# Evaluation of the last epoch
val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = \
model.evaluate_generator(test_batches, steps=val_steps)

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)
print('val_top_2_acc:', val_top_2_acc)
print('val_top_3_acc:', val_top_3_acc)

# Evaluation of the best epoch
model.load_weights('model.h5')

val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = \
model.evaluate_generator(test_batches, steps=val_steps)

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)
print('val_top_2_acc:', val_top_2_acc)
print('val_top_3_acc:', val_top_3_acc)

# Create a confusion matrix of the test images
test_labels = test_batches.classes

# Make predictions
predictions = model.predict_generator(test_batches, steps=val_steps, verbose=1)

# Declare a function for plotting the confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

cm_plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel','nv', 'vasc']

plot_confusion_matrix(cm, cm_plot_labels)

ModuleNotFoundError: No module named 'sklearn'